## Talybont farm Structure from Motion (SfM) workflow
----------------------

The following workflow demonstrates the full SfM process on a recently collected over a farm site near Talybont-on-Usk, Brecon. The flight was focussed on a sugar beet field from which to eventually derive species diversity information. The remit was to process surface reflectence imagery, ortho-rectify then mosaic  

This dataset consists of ~320 image captures ( X 5 including the 5 bands) using the Micasense Red Edge sensor aboard the Bramor PPX fixed wing platform.  

The following will run the SfM workflow though some issues made this more difficult, but those have been skipped in this demo and the data is the best pass by the UAV and photos are ready aligned for use.

The issues:

- Onbard GPS recording failure, hence a reliance on the camera GPS

- Three separate passes of the UAV as cloud cover changed

- Poor band alignment across all bands

Dependencies :

Micmac and the SfM scripts as avalaible on the QQ github. 

You will need to install this python module to run bash commands without constant exclamation marks

```bash
pip install bash_kernel

python -m bash_kernel.install
```

Whilst it is possible to use matplotlib to visualise some results in this notebook, QGIS 3 offers a more versatile platform for this purpose for the final datasets, so please ensure this is installed. Once installed, install the HCMGIS plugin to use various web-based base-layers for visualisation purposes. 

https://www.osgeo.org/projects/qgis/

Cloud compare or meshlab provide a useful tool to visualise point cloud results and can be found through the links below.

https://www.danielgm.net/cc/

http://www.meshlab.net/

Some extra info at the end of this workflow uses rsgislib, developed at Aberystwyth University (Bunting & Clewley, 2019) to finely register imagery using pixel and geospatial information.  

https://www.rsgislib.org/


The data:

- 2 folders of multispectral imagery RGB & RReNIR
- gps log (log.csv)

If you wish to skip through it all and just process, use this script, the inards of which are covered here. 

```bash

mspec_sfm.sh -e tif -a Forest -m PIMs -u '30 +north' -c Fraser log.csv

```

### Image Alignment

The image bands were aligned and outputted using the Micasense python lib. I have saved you the pain of finding a decent image for this which was one of the most problemtaic aspects of this dataset.


### SfM processng

Assuming you have already ```cd'd``` into the workspace containing RGB and RReNir. 

First we move the RGB data on which most processing is done.


In [2]:
mv RGB/*.tif $PWD

### Orientation

As with previous notebooks, we process the photo poses and produce a sparse cloud by first processing a relative orientation then the bundle adjustment of GPS etc.  

In [4]:
Orientation.sh -e JPG -u "30 +north" -c Fraser "30 +north" -t "log.csv" 

Next the images are orientated first in a relative coordinate system, then adjusted with the GPS data (Bundle adjustment). 

As the main onboard GPS failed to record, we only have the camera GPS data, which at best is accurate to within 3m. Hence we must specify an estimate of this to the function (```gpsAcc```) to enable the relative orientation and GPS position to be weighted appropriately. 

Had the main GPS worked, we could have specified 1m or less. 

### Evaluation of sparse cloud

We can look at the sparse point clouds for each dataset to check the result of the orientation in meshlab or cloud compare. This will be a .ply file Aperi_Ground_UTM.ply. 

As the data is in 16-bit colour depth, the RGB values will not display, hence the points will be white. You will still be able to see the structure is good nonetheless.


### Dense matching and output data

We will now use the ```PIMs``` function to generate DSM and individual ortho results. 

This will take some time to run for both datasets!

The results will be written to folders named 'OUTPUT'.

These may be opened in QGIS. 


In [ ]:
dense_cloud.sh -e tif -a PIMs -m Forest -i 1 -o 1

Now we move the RGB data out and get the Nir data

In [ ]:
mv *.tif RGB

mv OUTPUT/OrthoImage_geotif.tif OUTPUT/RGB.tif
rm -rf OUTPUT/OrthoImage_geotif.tif

mv RRENir/*.tif $PWD


Process the RReNir.... where ```-x 0``` indicates the DSM is not reprocessed, merely the ortho-imagery.

In [ ]:
dense_cloud.sh -e tif -a PIMs -m Forest -o 1 -x 0

mv *.tif RRENir

mv OUTPUT/OrthoImage_geotif.tif OUTPUT/RReNir.tif
rm -rf OUTPUT/OrthoImage_geotif.tif

The mosaicing was processed using the ```tawny``` function from MicMac. This uses radiometric equalisation by default and should produce satisfactory results and an evenly illuminated mosaic. 

Check the OUTPUT folder for mosaics and open in QGIS

This complete workflow can be achieved by one shell script cmd.

```bash

mspec_sfm.sh -e tif -a Forest -m PIMs -u '30 +north' -q 1 -d 1 -c Fraser log.csv

```

An aside - earlier versions of this workflow resulted in poor registration on RGB & the other bands. Since then an additional function in MicMac has arisen to get around this issue but I have retained this a it is useful. If images to not perfectly register, this handy function in RSGIS has proved useful on numerous occasions. 

https://www.rsgislib.org/rsgislib_imageregistration.html

...and below for the paper.

Bunting, P.J., Labrosse, F. & Lucas, R.M., 2010. A multi-resolution area-based technique for automatic multi-modal image registration. Image and Vision Computing, 28(8), pp.1203-1219.

It is likely good practice to do this prior to stacking the 3-band mosaics in most cases. 

The RReNir image will be registered to the RGB image.

**Before doing this switch kernels (kernels -> Change kernels) on the main menu at the top to one containing rsgislib.**

The following will use the basic registration, which is a simplified form of the proposed method

In [ ]:
from rsgislib import imageregistration

# The reference image
refImage = path.join(workdir,"SR2", "Pass1", "OUTPUT", "Moasaic.tif"
                     
# The floating image
fImage = path.join(workdir,"SR1", "Pass1", "OUTPUT", "Moasaic.tif" 
                   
# These are default values which work for this example 
pixelGap = 50
threshold = 0.4
window = 100
search = 5
stddevRef = 2
stddevFloat = 2
subpixelresolution = 4
                   
metric = imageregistration.METRIC_CORELATION
outputType = imageregistration.TYPE_RSGIS_IMG2MAP
                   
tiepoints = path.join(workdir, "tiepoints.txt")

# generate the tie points
imageregistration.basicregistration(refImage, fImage, pixelGap, threshold, 
                                    window, search, stddevRef, stddevFloat, 
                                    subpixelresolution, metric,
                                    outputType, tiepoints)

# output image                   
outRas = path.join(workdir,'RReNir_nnwp.tif'
                   
# warp the RReNir image
imageregistration.nnwarp(fImage, output, outRas, wkt, 0.08, gdalformat)